# Posterior sampling for Gaussian Mixture Model using Gibbs sampler

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import scipy.stats as st

import importlib
import GMM_gibbs as gmm

In [ ]:
def plot_points(zs,K,X,title):
    fig, ax = plt.subplots()
    datasets = []
    for i in range(K):
        # Assigned indices
        assigned_indices = (zs == i)
        datasets.append(X[assigned_indices, :])

    # Now let's put the scatter plots onto the scene.
    colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
    for j,data in enumerate(datasets):
        ax.scatter(data[:, 0], data[:, 1], color=colors[j])

    ax.set_title(title)

    plt.show()
    
def plot_logprob(iter_prob):
    fig, ax = plt.subplots()
    iteration_vec = range(0, len(iter_prob))
    ax.plot(iteration_vec, iter_prob)
    ax.set_title('Log Probability vs. Iterations')

    plt.show()

In [ ]:
def generate_test_data():
    """ This is the a test script used for me to generate test data"""
    # All of these will be assumed 2-D data
    # Let's generate a mixture of three gaussians
    u_0 = np.array([-2.0,-2.0])
    u_1 = np.array([0.0,2.0])
    u_2 = np.array([2.0,-2.0])

    # sigmas for each of the data
    sigmas = [1, 1, 1]
    points_per_cluster = 100

    # Now initialize the data variables
    data0 = np.random.randn(points_per_cluster,2)*sigmas[0] + u_0
    data1 = np.random.randn(points_per_cluster,2)*sigmas[1] + u_1
    data2 = np.random.randn(points_per_cluster,2)*sigmas[2] + u_2
    X = np.vstack((data0, data1, data2))

    # Now random init the cluster assignments
    rand_Y = np.random.randint(0, 3, points_per_cluster*3)
    return X, rand_Y

In [ ]:
X, Y = generate_test_data()

In [ ]:
X.shape, Y.shape

### Algorithm 1
- Sample mixture locations $\mu_k$ 
- Sample mixing coefficents $\pi_k$
- Sample cluster assignements $z_n$

Initialize variables

In [ ]:
K = 3
N = X.shape[0]

# GMM params
mus = np.array([np.random.rand() * np.ones(X.shape[1]) for k in range(K)])
# mus = np.array([1 * np.ones(K), 15 * np.ones(K)], dtype='float')
sigmas = np.array([np.diag(np.ones(X.shape[1])) for k in range(K)], dtype='float')
lambdas = np.array([np.linalg.inv(sigmas[k]) for k in range(K)])
# pis = np.array(0.5 * np.ones(K) )  # Mixing probs.
pis = np.ones(K)/ float(K)
zs = np.zeros([N])  # Assignments
for i in range(N):
    z = np.random.multinomial(n=N, pvals=pis).argmax()
    zs[i] = z
Y = zs.copy()
# Priors
alpha =np.ones(K)
pis = np.random.dirichlet(alpha)
mus0 = np.ones((K, X.shape[1]))
sigmas0 = np.array([np.diag(np.ones(X.shape[1])) for k in range(K)], dtype='float')
lambdas0 = np.array([np.linalg.inv(sigmas0[k]) for k in range(K)])

In [ ]:
plot_points(zs,K,X,title = 'Initial assignement')


### Gibbs sampler

In [ ]:
sigma = 1
normalizer = (1 / ((sigma ** 2) * np.sqrt(2 * np.pi)))
iter_prob = []
T = 50

In [ ]:
for it in range(T):
    # Sample from full conditional of assignment
    # z ~ p(z) \propto pi*N(y|pi)
    probs = np.zeros([N, K])

    for k in range(K):
        p = pis[k] * st.multivariate_normal.pdf(X, mean=mus[k], cov=sigmas[k])
        probs[:, k] = p

    # Normalize
    probs /= np.sum(probs, axis=1)[:, np.newaxis]

    # For each data point, draw the cluster assignment
    for i in range(N):
        z = np.random.multinomial(n=1, pvals=probs[i]).argmax()
        zs[i] = z

    # Sample from full conditional of cluster parameter
    # Assume fixed covariance => posterior is Normal
    # mu ~ N(mu, sigma)
    Ns = np.zeros(K, dtype='int')

    for k in range(K):
        # Gather all data points assigned to cluster k
        Xk = X[zs == k]
        Ns[k] = Xk.shape[0]

        # Covariance of posterior
        lambda_post = lambdas0[k] + Ns[k]*lambdas[k]
        cov_post = np.linalg.inv(lambda_post)

        # Mean of posterior
        left = cov_post
        right = lambdas0[k] @ mus0[k] + Ns[k]*lambdas[k] @ np.mean(Xk, axis=0)
        mus_post = left @ right

        # Draw new mean sample from posterior
        mus[k] = st.multivariate_normal.rvs(mus_post, cov_post)

    # Sample from full conditional of the mixing weight
    # pi ~ Dir(alpha + n)
    pis = np.random.dirichlet(alpha + Ns)
    
    # Calculate log-likelihood
    log_probs = np.zeros(X.shape[0])
    for i in range(0, X.shape[0]):
       # Calculate the sum of the lob probabilities
        clust = int(zs[i])
        d = np.linalg.norm(X[i, :] - mus[clust, :])
        log_probs[i] =  np.log(np.exp(-(d ** 2) / (2 * (sigma ** 2))) * normalizer)

    iter_prob.append(log_probs.sum())

for k in range(K):
    print('{} data in cluster-{}, mean: {}'.format(Ns[k], k, mus[k]))

In [ ]:
plot_points(zs,K,X,title = 'Final mixture assignement algorithm 1')

In [ ]:
plot_logprob(iter_prob)


### Algorithm 2: collapse $\pi$
- Sample mixture locations $\mu_k$ 
- Sample cluster assignement $z_n$

In [ ]:
importlib.reload(gmm)

In [ ]:
gs = gmm.MixGaussGibbsSampler(X, zs.astype('int'),sigma=1, lam=1, burn_in=50, lag=10)
gs.perform_gibbs_sampling(iterations=T)

### Compare mus algo1 and algo2

In [ ]:
print(gs.u_locations)

In [ ]:
for k in range(K):
    print(mus[k])